In [1]:
!pip install pandas python-docx pdfplumber openpyxl xlrd 

In [2]:
def clovaOCR(image_path: str) -> dict:
    """
    TODO: CLOVA OCR 호출(이미지 또는 스캔 PDF)
    content 스키마에 맞춰 반환
    """
    return {
        "pageCount": 1,
        "pages": {
            "page1": {
                "text": "",
                "tableCount": 1,
                "tables": {
                    "table1": {
                        "rows": [],
                        "rowCount": 0,
                        "colCount": 0
                    }
                }
            }
        }
    }


In [3]:
# 표처리 헬퍼
def clean_table(tbl):   
    return [
        [c for c in ("" if c is None else str(c).strip() for c in row) if c]
        for row in tbl
        if any("" if c is None else str(c).strip() for c in row)
    ]


In [ ]:
# 파일별 파싱 함수
from pathlib import Path
import pandas as pd
from docx import Document
import pdfplumber

CSV_EXT = {"csv"}
EXCEL_EXT = {"xls", "xlsx"}
IMAGE_EXT = {"jpg", "png"}
OTHER_EXT = {"pdf", "docx"}

def handle_csv(path: str):
    for enc in ["cp949", "euc-kr", "utf-8"]:
        try:
            df = pd.read_csv(path, encoding=enc)
            return {"kind": "CSV", "ext": "csv", "dataframe": df}
        except UnicodeDecodeError:
            continue
    raise ValueError("인코딩 실패: cp949, euc-kr, utf-8 encoding을 지원합니다")

def handle_excel(path: str, ext: str):
    df = pd.read_excel(path)
    return {"kind": "EXCEL", "ext": ext, "dataframe": df}

def handle_image(path: str, ext: str):
    ocr_output = clovaOCR(path)
    return {"kind": "IMAGE", "ext": ext, "content": ocr_output}

def clean_table(tbl):
    return [
        [c for c in ("" if c is None else str(c).strip() for c in row) if c]
        for row in tbl
        if any("" if c is None else str(c).strip() for c in row)
    ]

def handle_pdf(path: str, ext: str):
    PDF_PAGE_MIN_CHARS = 30
    PDF_PASS_RATIO = 0.7

    with pdfplumber.open(path) as pdf:
        total_pages = len(pdf.pages)
        valid_page_count = 0
        pages_data = {}

        for i, page in enumerate(pdf.pages):
            t = page.extract_text() or ""
            clean_t = " ".join(t.split()).strip()
            if len(clean_t) >= PDF_PAGE_MIN_CHARS: valid_page_count += 1
            raw = page.extract_tables() or []
            tables = {f"table{ti+1}": (lambda rows: {"rows": rows, "rowCount": len(rows), "colCount": max((len(r) for r in rows), default=0)})(clean_table(tbl)) for ti, tbl in enumerate(raw)}
            pages_data[f"page{i+1}"] = {"text": clean_t, "tableCount": len(raw), "tables": tables}

        pass_ratio = valid_page_count / max(total_pages, 1)

    if pass_ratio >= PDF_PASS_RATIO: return {"kind":"PDF","ext":ext,"mode":"text","pass_ratio":pass_ratio,"content":{"pageCount":total_pages,"pages":pages_data}}
    else:
        ocr_output = clovaOCR(path)
        return {"kind": "PDF", "ext": ext, "mode": "ocr", "content": ocr_output}

def handle_docx(path: str, ext: str):
    doc = Document(path)
    paragraphs = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    tables = {}
    for i, table in enumerate(doc.tables, start=1):
        rows = [[cell.text for cell in row.cells] for row in table.rows]
        tables[f"table{i}"] = {
            "rows": rows,
            "rowCount": len(rows),
            "colCount": max((len(r) for r in rows), default=0),
        }
    pages = {
        "page1": {
            "text": "\n".join(paragraphs),
            "tableCount": len(doc.tables),
            "tables": tables,
        }
    }
    return {"kind": "DOCX", "ext": ext, "content": {"pageCount": 1, "pages": pages}}


In [5]:
# 파일 형식별 조건부 함수구동
def handle_file(path: str):
    ext = Path(path).suffix.lower().lstrip(".")
    if ext in CSV_EXT: return handle_csv(path)
    if ext in EXCEL_EXT: return handle_excel(path, ext)
    if ext in IMAGE_EXT: return handle_image(path, ext)
    if ext == "pdf": return handle_pdf(path, ext)
    if ext == "docx": return handle_docx(path, ext)
    raise ValueError(f"지원하지 않는 파일 형식입니다: {ext}")

In [6]:
# docx 테스트
path = "C:/Users/User/Downloads/현대협력_ESG_진단_산출물_진단서_위험군보고서.docx"
test_result = handle_file(path)
test_result

{'kind': 'DOCX',
 'ext': 'docx',
 'content': {'pageCount': 1,
  'pages': {'page1': {'text': '공급망 ESG 진단 산출물\n(진단서 + 위험군 보고서)\n※ 이 문서는 실제 제출용이 아닌 시연용(임시) 예시입니다. 실제 서비스에서는 업종/규모/증빙 업로드 결과에 따라 항목·결과가 자동 생성됩니다.\n목차\n위험군 보고서\n진단서\n1. 위험군 보고서\n1.1 등급\n종합 등급 : B (중 위험군)\n판정 근거(요약): 필수 항목 중 미충족 3건 및 미확인/누락 다수 존재\n1.2 결과(10줄 이상, 세부)\n정성 체크리스트 25개 중 충족 6개, 부분충족 11개, 미충족 3개, 미확인 5개로 집계됨(샘플).\n인권·노동 영역에서 차별·괴롭힘 금지 및 신고채널 운영이 미충족으로 확인되어 인권 리스크가 상향됨.\n강제노동 금지 절차는 일부 존재하나(부분충족), 해외/이주노동자 관련 증빙이 없어 검증 강도가 낮음.\n근로시간 관리는 일부 기록으로 확인되나, 초과근로 통제(사전 승인/상한 관리) 체계가 불충분함.\n안전보건 영역에서 산업재해·아차사고 보고/재발방지 프로세스가 미충족으로 확인되어 중대사고 예방 체계가 취약함.\n위험성평가는 일부 수행 기록이 있으나, 개선조치 이행 증빙(CAPA 완료 증빙)이 부족하여 효과 검증이 어려움.\n외주/협력 인력의 안전교육 이수율 데이터가 누락되어, 현장 작업자 전체에 대한 안전 통제가 확인되지 않음.\n환경 영역은 폐기물 인계서로 기본 준수는 확인되지만, 인허가·법정 측정 성적서가 미제출되어 규제 리스크를 정량 평가하기 어려움.\n온실가스(Scope 1+2) 및 용수 사용량 등 핵심 환경 데이터가 누락되어, 고객사 요구 또는 공시 대응에 제약이 발생할 수 있음.\n윤리·컴플라이언스 영역에서 부패방지 정책은 있으나, 승인/기록 절차가 미흡하고 교육 이수율(60%)이 낮아 잔존 리스크가 존재함.\n정보보안 영역에서 보안사고 대응 및 백업 정책이 미충족이며, 

In [7]:
# pdf 테스트
path = "C:/Users/User/Downloads/현대협력_ESG_진단_산출물_진단서_위험군보고서.pdf"
test_result = handle_file(path)
test_result

{'kind': 'PDF',
 'ext': 'pdf',
 'mode': 'text',
 'content': {'pageCount': 10,
  'pages': {'page1': {'text': '공급망 ESG 진단 산출물\n(진단서 + 위험군 보고서)\n1',
    'tableCount': 0,
    'tables': {}},
   'page2': {'text': '원청사명 현대중공업\n협력사명 현대협력\n작성일 2026-01-14\n문서버전 v0.1\n산출물 구성 1) 위험군 보고서 2) 진단서\n비고 본 문서는 데모/샘플 데이터로 작성됨\n※ 이 문서는 실제 제출용이 아닌 시연용(임시) 예시입니다. 실제 서비스에서는 업종/규모/증빙 업로드\n결과에 따라 항목·결과가 자동 생성됩니다.\n목차\n1. 위험군 보고서\n2. 진단서\n2',
    'tableCount': 1,
    'tables': {'table1': {'rows': [['현대중공업'],
       ['원청사명'],
       ['현대협력'],
       ['협력사명'],
       ['2026-01-14'],
       ['작성일'],
       ['v0.1'],
       ['문서버전'],
       ['1) 위험군 보고서 2) 진단서'],
       ['산출물 구성'],
       ['본 문서는 데모/샘플 데이터로 작성됨'],
       ['비고']],
      'rowCount': 12,
      'colCount': 1}}},
   'page3': {'text': '1. 위험군 보고서\n1.1 등급\n종합 등급 : B\n(중 위험군)\n종합 점수 68 / 100\n정성 점수 38 / 60\n정량 점수 30 / 40\n판정 근거(요약): 필수 항목 중 미충족 3 건 및 미확인/누락 다수 존재\n1.2 결과(10 줄 이상, 세부)\n1. 정성 체크리스트 25개 중 충족 6개, 부분충족 11개, 미충족 3개, 미확인 5개로 집계됨(샘플).\n2. 인권·노동 영역에

In [8]:
# csv 테스트
path = "C:/Users/User/Downloads/Steel_industry_data.csv"
test_result = handle_file(path)
test_result

{'kind': 'CSV',
 'ext': 'csv',
 'dataframe':                   癤풼ate  Usage_kWh  Lagging_Current_Reactive.Power_kVarh  \
 0      01/01/2018 00:15       3.17                                  2.95   
 1      01/01/2018 00:30       4.00                                  4.46   
 2      01/01/2018 00:45       3.24                                  3.28   
 3      01/01/2018 01:00       3.31                                  3.56   
 4      01/01/2018 01:15       3.82                                  4.50   
 ...                 ...        ...                                   ...   
 35035  31/12/2018 23:00       3.85                                  4.86   
 35036  31/12/2018 23:15       3.74                                  3.74   
 35037  31/12/2018 23:30       3.78                                  3.17   
 35038  31/12/2018 23:45       3.78                                  3.06   
 35039  31/12/2018 00:00       3.67                                  3.02   
 
        Leading_Current_Reacti

In [9]:
# xlsx 테스트
path = "C:/Users/User/Downloads/Steel_industry_data_xlsx.xlsx"
test_result = handle_file(path)
test_result

{'kind': 'EXCEL',
 'ext': 'xlsx',
 'dataframe':                    date  Usage_kWh  Lagging_Current_Reactive.Power_kVarh  \
 0      01/01/2018 00:15       3.17                                  2.95   
 1      01/01/2018 00:30       4.00                                  4.46   
 2      01/01/2018 00:45       3.24                                  3.28   
 3      01/01/2018 01:00       3.31                                  3.56   
 4      01/01/2018 01:15       3.82                                  4.50   
 ...                 ...        ...                                   ...   
 35035  31/12/2018 23:00       3.85                                  4.86   
 35036  31/12/2018 23:15       3.74                                  3.74   
 35037  31/12/2018 23:30       3.78                                  3.17   
 35038  31/12/2018 23:45       3.78                                  3.06   
 35039  31/12/2018 00:00       3.67                                  3.02   
 
        Leading_Current_Rea

In [10]:
# jpg 테스트
path = "C:/Users/User/Downloads/성광벤드 ISO 45001_page-0001.jpg"
test_result = handle_file(path)
test_result

{'kind': 'IMAGE',
 'ext': 'jpg',
 'content': {'pageCount': 1,
  'pages': {'page1': {'text': '',
    'tableCount': 1,
    'tables': {'table1': {'rows': [], 'rowCount': 0, 'colCount': 0}}}}}}